In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_3.csv")

In [3]:
df.head()

,timestamp_local,temp,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,430,1782.5,60.7,2.7,533.7,377.00,253.3


In [5]:
data = df[['co']]

In [6]:
from sklearn.preprocessing import MinMaxScaler
values = data.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [7]:
scaled = scaled.reshape((20978,))

In [8]:
scaled.shape

(20978,)

In [9]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [10]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

X_max = X_train.max()
X_min = X_train.min()

In [11]:
batchSize = 16
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [12]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

C:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 166, 8)              │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1328)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 168)                 │         223,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,304 (872.28 KB)

 Trainable params: 223,304 (872.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0024 - root_mean_squared_error: 0.0482 - val_loss: 0.0019 - val_root_mean_squared_error: 0.0437 - learning_rate: 0.0010
Epoch 2/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0017 - root_mean_squared_error: 0.0415 - val_loss: 0.0018 - val_root_mean_squared_error: 0.0420 - learning_rate: 0.0010
Epoch 3/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0015 - root_mean_squared_error: 0.0387 - val_loss: 0.0017 - val_root_mean_squared_error: 0.0409 - learning_rate: 0.0010
Epoch 4/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0015 - root_mean_squared_error: 0.0384 - val_loss: 0.0017 - val_root_mean_squared_error: 0.0408 - learning_rate: 0.0010
Epoch 5/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0013 - root_mean_squared_error: 0.0366 - val_loss: 0.0018 - val_root_mean_squared_error: 0.0421 - learning_rate: 0.0010
Epoch 6/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0016 - root_mean_squared_err

In [13]:
forecast = model.predict(X_test)

130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [14]:
poll = np.array(df["co"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 13.474933


In [17]:
# Calculate MAPE
mape = np.mean(np.abs((y_test_true - testPredict) / y_test_true)) * 100
print("Test MAPE =", mape)

Test MAPE = 1.8390018492937088


In [18]:
# Calculate accuracy as 1 - MAPE
accuracy = 100 - mape
print("Accuracy =", accuracy, "%")

Accuracy = 98.16099815070629 %


In [19]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae = mean_absolute_error(y_test_true, testPredict)
print("Test MAE =", mae)

Test MAE = 6.8339844


In [20]:
forecast

array([[0.03501679, 0.03641824, 0.03720342, ..., 0.04283508, 0.04362013,
        0.04333312],
       [0.02743972, 0.02615397, 0.02588571, ..., 0.04602275, 0.03586999,
        0.02957134],
       [0.06150617, 0.05664637, 0.05146141, ..., 0.05245339, 0.05557515,
        0.05757346],
       ...,
       [0.11328793, 0.13090402, 0.14671513, ..., 0.06056726, 0.0728304 ,
        0.08189704],
       [0.11126898, 0.07729384, 0.06238278, ..., 0.15673962, 0.14313187,
        0.11651459],
       [0.01114965, 0.01070249, 0.01017707, ..., 0.01714508, 0.01759373,
        0.01708926]], dtype=float32)

In [21]:
testPredict

array([[347.89035, 348.3689 , 348.63702, ..., 350.56006, 350.82812,
        350.7301 ],
       [345.30304, 344.864  , 344.7724 , ..., 351.64853, 348.1817 ,
        346.0309 ],
       [356.9356 , 355.27615, 353.50565, ..., 353.8444 , 354.91037,
        355.5927 ],
       ...,
       [374.61737, 380.6327 , 386.03165, ..., 356.615  , 360.80246,
        363.8984 ],
       [373.92798, 362.32657, 357.23495, ..., 389.45468, 384.80807,
        375.71918],
       [339.74054, 339.58783, 339.40842, ..., 341.78778, 341.94098,
        341.7687 ]], dtype=float32)

In [22]:
y_test_true

array([[349.7817 , 351.3044 , 352.82712, ..., 342.24966, 342.63818,
        343.02667],
       [345.72116, 345.3452 , 344.96295, ..., 346.6862 , 343.72223,
        340.7583 ],
       [358.20355, 354.87616, 351.5488 , ..., 350.95975, 351.88718,
        352.81458],
       ...,
       [379.41483, 378.6566 , 377.90463, ..., 350.89084, 355.1331 ,
        359.37534],
       [374.95325, 360.69125, 356.10437, ..., 372.4154 , 367.01392,
        358.473  ],
       [339.21054, 339.22308, 338.60272, ..., 337.6001 , 337.2116 ,
        337.08002]], dtype=float32)

In [23]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [24]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 4.0151
RMSE for segment 2: 5.5160
RMSE for segment 3: 6.5950
RMSE for segment 4: 7.6733
RMSE for segment 5: 8.7613
RMSE for segment 6: 9.2883
RMSE for segment 7: 9.6046
RMSE for segment 8: 9.9867
RMSE for segment 9: 10.2688
RMSE for segment 10: 10.5310
RMSE for segment 11: 10.7187
RMSE for segment 12: 10.7949
RMSE for segment 13: 10.9267
RMSE for segment 14: 10.9947
RMSE for segment 15: 11.1204
RMSE for segment 16: 11.2943
RMSE for segment 17: 11.2115
RMSE for segment 18: 11.1915
RMSE for segment 19: 11.1597
RMSE for segment 20: 11.2131
RMSE for segment 21: 11.1007
RMSE for segment 22: 11.0946
RMSE for segment 23: 10.8727
RMSE for segment 24: 10.7441
RMSE for segment 25: 10.8406
RMSE for segment 26: 10.5827
RMSE for segment 27: 10.4507
RMSE for segment 28: 10.3960
RMSE for segment 29: 10.6032
RMSE for segment 30: 10.4630
RMSE for segment 31: 10.4235
RMSE for segment 32: 10.7958
RMSE for segment 33: 11.4160
RMSE for segment 34: 11.8277
RMSE for segment 35: 11.9390
RM

In [25]:
model.save("co_1dcnn.keras")